In [ ]:
import misfunciones

### Lab 7 - Optimizando y acelerando nuestros algoritmos 🏎🏁

En la última parte del lab 6, vimos cómo efectuar una búsqueda _difusa_ de k-meros en una secuencia empleando la distancia de Hamming, que calculábamos en la función `distancia(kmero_1, kmero_2)`:

In [ ]:
def distancia(kmero_1, kmero_2):
    contador_difs = 0
    for c, b in zip(kmero_1, kmero_2):
        if c != b:
            contador_difs += 1
    return contador_difs

Usando `distancia(kmero_1, kmero_2)`, pudimos implementar otra función `cuenta_kmero_aprox(secuencia, kmero, d)` que devolvía cuántas veces aparecía un k-mero en una secuencia permitiendo variaciones en `d` letras:

In [ ]:
def cuenta_kmero_aprox(secuencia, kmero, d):
    cuenta = 0
    for i in range(len(secuencia) - len(kmero) + 1):
        if distancia(secuencia[i:i+len(kmero)], kmero) <= d:
            cuenta += 1
        if distancia(secuencia[i:i+len(kmero)], misfunciones.inv_comp(kmero)) <= d:
            cuenta += 1
    return cuenta

Finalmente, generamos un _mapa de frecuencias_ para k-meros permitiendo al menos variaciones en `d` letras con la función:

In [ ]:
def kmeros_frecuentes_aprox(secuencia, k, d):
    freq = {}
    n = len(secuencia)
    todos_kmeros = list(["".join(kmero) for kmero in itertools.product("ACGT", repeat=k)])
    for kmero in todos_kmeros:
        frec = cuenta_kmero_aprox(secuencia, kmero, d)
        freq[kmero] = frec
    return freq

In [ ]:
def lee_genoma(ruta_fichero):
  fd = open(ruta_fichero)
  genome = fd.read()
  fd.close()
  return genome

In [ ]:
ec_genoma = lee_genoma('../data/E-coli.txt')

In [ ]:
%time resultados  = kmeros_frecuentes_aprox(ec_genoma[3923620:3923620+500], 9, 1) # tarda mucho! ¿Cómo lo arreglamos?

Sin embargo, también pudimos comprobar que este algoritmo tardaba bastante en ejecutarse, ya un requisito era generar previamente los $4^{k}$ k-meros posibles. Una pregunta que quedó en el aire era cómo podíamos acelerar este algoritmo: vamos a verlo. 

## Optimizando
La optimización en este caso la vamos a conseguir considerando sólo el grupo de k-meros que tengan una distancia `d` o inferior a cualquier k-mero que aparezca en la secuencia. A estos grupos, los denominaremos `d-vecindarios`.

Por ejemplo, el 1-vecindario (vecinos inmediatos) del 3-mero `ACG` es: `{ACG, ACG CCG GCG TCG AAG AGG ATG ACA ACC ACT}` (nótese que se incluye el "vecino" originario del vecindario). 

Como ves, el tamaño del d-vecindario de un k-mero dependerá del número de símbolos del alfabeto y la longitud del k-mero.

__Pregunta: ¿Cuántos vecinos tendrá el vecindario de un k-mero (para el alfabeto del ADN)?__


Vamos a implementar la función `vecinos_directos(kmero, alfabeto)`, que nos devolverá el conjunto de vecinos directos para un k-mero y un alfabeto:

In [ ]:
def vecinos_directos(kmero, alfabeto):
    #Tu código aquí

In [ ]:
vecinos_directos('ACG', ('A', 'C', 'T', 'G'))

### d-vecindarios
La función que hemos implementado genera el 1_vecindario (vecinos directos) para un k-mero, ¡pero recuerda que la búsqueda difusa puede permitir más modificaciones!

__Pregunta: Piensa cómo generarías el d-vecindario de un k-mero dado. Acaba de implementar la función que te doy en la siguiente celda.__


In [ ]:
def vecindario(kmero, alfabeto, d):
    #Tu código aquí 

In [ ]:
vecindario('ACGT', ('A', 'C', 'T', 'G'), 2)

### Sólo los vecinos
Ahora que ya sabemos encontrar el d-vecindario de un k-mero, podemos modificar nuestro algoritmo para generar mapas de frecuencias siguiendo nuestro razonamiento inicial. 

__Pregunta: Modifica la función kmeros_frecuentes_aprox para que considere sólo los k-meros que sean miembros de algún vecindario de todos los k-meros que aparezcan en la secuencia. Mide el tiempo que tarda en ejecutarse con la magia `%time` y compárala con la que obtuviste con la versión de "fuerza bruta" del algoritmo en el anterior lab.__

In [ ]:
def kmeros_frecuentes_aprox_vecindarios(secuencia, k, d, alfabeto):
    #Tu código aquí

In [ ]:
%time resultados = kmeros_frecuentes_aprox_vecindarios(ec_genoma[3923620:3923620+500], 9, 1, ('A', 'C', 'T', 'G'))

In [ ]:
len(resultados)

### Encontrado cajas DnaA en E.Coli (pero más rápido!)
Finalmente, podemos analizar el mapa de frecuencias para sacar las cajas DnaA. ¡Comprueba que los resultados son iguales a los obtenidos en el anterior lab! 

In [ ]:
valor_max = max(resultados.values())
top_kmeros = []
for k,v in resultados.items():
    if v == valor_max:
        top_kmeros.append(k)

print(top_kmeros, valor_max)

In [ ]:
def agrupa_resultados(top_kmeros):
    grupos = []
    while len(top_kmeros) > 0:
        kmero = top_kmeros.pop()
        grupo_temp = [kmero]
        for i in range(len(top_kmeros)):
            if distancia(kmero, top_kmeros[i]) <= 1 or kmero == misfunciones.inv_comp(top_kmeros[i]) or distancia(misfunciones.inv_comp(kmero), top_kmeros[i]) <= 1:
                grupo_temp.append(top_kmeros[i])

        top_kmeros = [kmero for kmero in top_kmeros if kmero not in grupo_temp]
        grupos.append(sorted(grupo_temp)) # ordenamos cada grupo lexicográficamente para ver sus diferencias.
    
    return sorted(grupos, key=lambda kmeros: kmeros[0]) # mantenemos el orden lexicográfico en la lista de grupos

In [ ]:
agrupa_resultados(top_kmeros)

### Conclusión
Como ves, muchas veces el proceso de optimización de un algoritmo implica identificar repeticiones excesivas de líneas de código y aplicar ciertas _heurísticas_ para guiar la búsqueda.

__Pregunta: ¿Es la diferencia en tiempos la esperada? ¿Cómo podrías haber "previsto" cuál sería la reducción en el tiempo de ejecución? Calcúlala y comprueba si lo obtenido empíricamente se ajusta a tu predicción.__